---
Author: Viswambhar Yasa
Date: 09-01-2024
---

# Taylor series on a classification task (How it explains the uncertainity?)

**Description:**  
Taylor series expansion is visualized on a f(x)=sin(x) and evaluated it for different number of steps

In the classification task, gradients are employed to determine the direction of each data point, guiding them towards the nearest boundary condition. This process is visualized to enhance understanding of how each point relates to the defined boundaries.

**Contact Information:**  
Email: [yasa.viswambhar@gmail.com](mailto:yasa.viswambhar@gmail.com)

## Additional Comments
We used taylor series to optimise the graident directory pointing to the nearest boundary.

In [1]:
import sympy as sp
import numpy as np
import plotly.io as pio
from scipy.optimize import root
import plotly.graph_objects as go

In [3]:
# Define the functions for sin(x) and Taylor approximation
def func(x):
    return np.sin(x)

def taylor_approximation(x, n):
    return sum([(-1)**i * x**(2 * i + 1) / np.math.factorial(2 * i + 1) for i in range(n)])
fontsize=35
# Values for x
x = np.linspace(-2*np.pi, 2*np.pi, 400)

# Creating a Plotly figure
fig = go.Figure()

# Adding the sin(x) function
fig.add_trace(go.Scatter(x=x, y=func(x), mode='lines', name='sin(x)', line=dict(color='blue', dash='solid')))

# Taylor Series Approximations
taylor_terms = [1, 3, 5, 7, 9]
colors = ['green', 'red', 'purple', 'orange', 'brown']

# Adding the Taylor Series Approximations with dashed lines
for n, color in zip(taylor_terms, colors):
    taylor_y = taylor_approximation(x, n)
    fig.add_trace(go.Scatter(x=x, y=taylor_y, mode='lines', name=f'Taylor Series ({n} terms)',
                             line=dict(color=color, dash='dash')))

# Update layout with increased font size for legend, ticks, and axis labels
fig.update_layout(
    title='Taylor Series Approximation  ',
    xaxis=dict(
        title='x',
        title_font=dict(size=fontsize),
        tickfont=dict(size=fontsize-10)
    ),
    yaxis=dict(
        title='f(x)= sin(x)',
        title_font=dict(size=fontsize),
        tickfont=dict(size=fontsize-10),
        range=[-2, 2]
    ),
    legend=dict(
        y=-0.2,
        x=0.5,
        xanchor='center',
        orientation='h',
        font=dict(size=fontsize)
    ),
    margin=dict(b=100),  # margin to ensure legend is visible
    font=dict(color='black',size=fontsize)  # dark black color for better readability
)

# Show the plot

fig.write_image("./figures/taylor_series_sinx.png", scale=2,width=2400,height=800)

fig.show()

In [4]:
# Define the function to plot

def f(x, y):
    return -1 + 2 * np.exp(-((x - 1)**2 + y**2)) + 2 * np.exp(-((x + 1)**2 + y**2))


def gradient(x, y):
    df_dx = -4 *((x - 1) * np.exp(-((x - 1)**2 + y**2))+ (x+1) * np.exp(-((x + 1)**2 + y**2)))
    df_dy = -4 *y*( np.exp(-((x - 1)**2 + y**2)) + np.exp(-((x + 1)**2 + y**2)))
    return np.array([df_dx, df_dy])

def equation(vars):
    x, y = vars
    return [f(x, y),0]

# Generate x and y values
x = np.linspace(-2.5, 2.5, 100)
y = np.linspace(-1.5, 1.5, 100)
X, Y = np.meshgrid(x, y)

# Calculate the function values for each point
Z = f(X, Y)

Z_positive = (Z > 0.4) & (Z<=1)
Z_negative=(Z < -0.45) & (Z>-0.65)
points_xp = X[Z_positive]
points_yp = Y[Z_positive]
points_xn = X[Z_negative]
points_yn = Y[Z_negative]
random_indices1 = np.random.choice(len(points_xp), size=50, replace=False)
random_indices2 = np.random.choice(len(points_xn), size=100, replace=False)

# Select the corresponding points
selected_points_xp = points_xp[random_indices1]
selected_points_yp = points_yp[random_indices1]
selected_points_xn = points_xn[random_indices2]
selected_points_yn = points_yn[random_indices2]

grad_pointxp=selected_points_xp[25]
grad_pointyp=selected_points_yp[25]
positive_grad=gradient(grad_pointxp,grad_pointyp)
grad_pointxn=selected_points_xn[25]
grad_pointyn=selected_points_yn[25]
negative_grad=gradient(grad_pointxn,grad_pointyn)


print("postive grad",positive_grad,"negative grad",negative_grad)

postive grad [0.6125023 0.0467864] negative grad [-0.19116716 -0.92344662]


In [5]:
# Initial guess
initial_guess = [-0.75, 0.3]

# Solve the equation
result = root(equation, initial_guess)
if result.success:
    root_values = result.x
    root_grad=gradient(root_values[0],root_values[1])
    print("Roots:", root_values)
    print("Roots Gradiant",root_grad)

Roots: [1.44007973 0.70895851]
Roots Gradiant [-0.89272033 -1.41791701]


In [6]:
# Define the variables
x2, y2 = sp.symbols('x,y')
# Define the function
f1 = -1 + 2 * sp.exp(-((x2 - 1)**2 + y2**2)) + 2 * sp.exp(-((x2 + 1)**2 + y2**2))

x0, y0 = 0.25,0.25
#x0 = grad_pointxp
#y0= grad_pointyp
order = 1 # Specify the order of the expansion
 
# Compute the Taylor series
taylor_expansion = f1.series(x2, x0, order).removeO().series(y2, y0, order).removeO()

# Print the result
print(taylor_expansion)

0.464346207446369


In [7]:
f1

2*exp(-y**2 - (x - 1)**2) + 2*exp(-y**2 - (x + 1)**2) - 1

In [8]:
approximation=f(grad_pointxp,grad_pointyp)+gradient(grad_pointxp,grad_pointyp)*(grad_pointxp-root_values[0],grad_pointyp-root_values[1])

In [9]:

fontsize=35
fig = go.Figure(data=go.Contour(
x=x, y=y, z=Z,
#contours_coloring='lines',
        line_width=2,
line_smoothing=0.85,
contours=dict(
        start=-1, end=1, size=0.15,     coloring ='none',
            showlabels = True, # show labels on contours
            labelfont = dict( # label font properties
                size = fontsize-10,
                color = 'black',
            )),
        name='contour lines'

))
contour_lines = fig.data[0]['line']
contour_lines['color']='grey'
#contour_lines['labelfont']['color']='red'
# Add scatter plots for selected positive and negative points
fig.add_trace(go.Contour(
x=x, y=y, z=Z,
#contours_coloring='lines',
line_color='orange',
        line_width=5,
line_smoothing=0.85,
line_dash='dashdot',
   contours=dict(
        start=0, end=0, size=1,
            coloring ='none',
            showlabels = True, # show labels on contours
            labelfont = dict( # label font properties
                size = fontsize-5,
                color = 'orange',
            )),
            name='decision boundary'
))

#fig.add_trace(go.Contour(
#x=x, y=y, z=Z,
#contours_coloring='lines',
#line_color='grey',
#        line_width=0.5,
#line_smoothing=0.85,
#line_dash='dash',
#   contours=dict(
#        start=-1, end=1, size=0.25,
#            coloring ='none',
#            showlabels = True, # show labels on contours
#            labelfont = dict( # label font properties
#                size = 8,
#                color = 'grey',
#            )),
#            name='contour lines'
#))

fig.add_trace(go.Scatter(
    x=selected_points_xp.flatten(), y=selected_points_yp.flatten(),
    mode='markers', name='Positive',
    #circle-open-dot
    marker=dict(symbol="circle",color='blue', size=fontsize-10,opacity=0.70)
))

fig.add_trace(go.Scatter(
    x=selected_points_xn.flatten(), y=selected_points_yn.flatten(),
    mode='markers', name='Negative',
    #"x-open-dot"
    marker=dict(symbol="x",color='green', size=fontsize-10,opacity=0.70)
))

fig.add_trace(go.Scatter(
    x=[root_values[0]], y=[root_values[1]],
    mode='markers', name='f(x,y) nearest roots',
    #circle-open-dot
    marker=dict(symbol="diamond",color='black', size=10),
))

fig.add_trace(go.Scatter(
    x=[grad_pointxp,root_values[0]], y=[grad_pointyp,root_values[1]],
    mode='lines', name='(x-x0,y-y0)',
    line=dict(dash="dash",color='red',width=4)
    #circle-open-dot
))

fig.update_layout(
    annotations=[
        go.layout.Annotation(dict(
            x= positive_grad[0],
            y=positive_grad[1],
            showarrow=True,
            axref="x", ayref='y',
            ax=grad_pointxp,
            ay=grad_pointyp,
            text='',
            arrowhead=3,
            arrowsize=1.5,
            arrowwidth=4,
            arrowcolor='black'
        )),
        go.layout.Annotation(dict(
            x=root_grad[0],
            y=-root_grad[1],
            showarrow=True,
            axref="x", ayref='y',
            ax=root_values[0],
            ay=root_values[1],
            text='',
            arrowhead=3,
            arrowsize=1.5,
            arrowwidth=4,
            arrowcolor='black'
        )),
        go.layout.Annotation(dict(
            x=approximation[0],
            y=approximation[1],
            showarrow=True,
            axref="x", ayref='y',
            ax=grad_pointxp,
            ay=grad_pointyp,
            text='',
            arrowhead=3,
            arrowsize=1.5,
            arrowwidth=6,
            arrowcolor='red'
        ))
    ]
)


fig.update_layout(title = "f(x,y)="+str(f1),title_x=0.5,
    xaxis=dict(title='X',title_font=dict(size=fontsize),
        tickfont=dict(size=fontsize-10),),
    yaxis=dict(title='Y',
               title_font=dict(size=fontsize),
        tickfont=dict(size=fontsize-10),),
    legend=dict(
        y=-0.2,
        x=0.5,
        xanchor='center',
        orientation='h',
        font=dict(size=fontsize)
    ),
    margin=dict(b=100),  # margin to ensure legend is visible
    font=dict(color='black',size=fontsize),  # dark black color for better readability,
    showlegend=True
)

fig.write_image("./figures/taylor_series.png", scale=2,width=2400,height=1000)

# Show the plot
fig.show()